# Feature engineering & selection

**Jupyter Kernel**:


* If you are in SageMaker Studio, make sure that you use the **PyTorch 1.10 Python 3.8 CPU Optimized** environment.
* Make sure that you are using one of the following instance types: `ml.m5.large` or `ml.g4dn.xlarge`.

**Run All**: 

* If you are in SageMaker Studio, you can choose the **Run All Cells** from the **Run** tab dropdown menu to run the entire notebook at once.

In [ ]:
# Install dependencies that will be used in this notebook.
!pip3 install -r ./utils/requirements.in -q

This solution relies on a config file to run the provisioned AWS resources. Run the cell below to generate that file.

In [ ]:
import boto3
import os
import json

In [ ]:
client = boto3.client('servicecatalog')
cwd = os.getcwd().split('/')
i= cwd.index('S3Downloads')
pp_name = cwd[i + 1]
pp = client.describe_provisioned_product(Name=pp_name)
record_id = pp['ProvisionedProductDetail']['LastSuccessfulProvisioningRecordId']
record = client.describe_record(Id=record_id)

keys = [ x['OutputKey'] for x in record['RecordOutputs'] if 'OutputKey' and 'OutputValue' in x]
values = [ x['OutputValue'] for x in record['RecordOutputs'] if 'OutputKey' and 'OutputValue' in x]
stack_output = dict(zip(keys, values))

with open(f'/root/S3Downloads/{pp_name}/stack_outputs.json', 'w') as f:
    json.dump(stack_output, f)

In [ ]:
sagemaker_config = json.load(open("stack_outputs.json"))

SOLUTION_BUCKET = sagemaker_config["SolutionS3Bucket"]
AWS_REGION = sagemaker_config["AWSRegion"]
SOLUTION_NAME = sagemaker_config["SolutionName"]
AWS_S3_BUCKET = sagemaker_config["S3Bucket"]

KEY_YIELD_CURVE = "data/raw/yield_curve_field_dt.csv"
SPATIAL_FILES_KEY = "data/spatial-files"
FIPS_STATS_KEY = "data/fips-stats/fips_county_stats.csv"
FIPS_POLYGONS_KEY = "data/fips-stats/geojson-counties-fips.json"
SENTINEL_2_SHAPEFILE_KEY = "data/sentinel-2-shapefiles"
CROPS_MASK_KEY = "data/crop_mask/raw"
REQUEST_MANIFESTS_KEY = "request_manifests/"

### Set up the environment

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import datetime
import seaborn as sns
import boto3
import io
import os
import s3fs
import itertools as it
import time
import spyndex
import dask.dataframe as dd
from typing import List
import warnings
import cmapy
import random

from sklearn.preprocessing import RobustScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor

from matplotlib.cm import get_cmap
from matplotlib.colors import Colormap, CenteredNorm, rgb2hex
import matplotlib.pyplot as plt
import plotly.express as px

import xgboost
import shap

import sagemaker

from utils.helper_functions import (
    download_s3_folder,
    convert_dates,
    heatmap_pandas,
    centered_gradient,
    global_shap_importance
    
)

warnings.simplefilter('ignore')

%matplotlib inline

In [ ]:
# Define a few variables to use throughout the notebook

YEAR = 2018 # crop year
CROP_TYPE = 'corn' # crop type
CROP_REGION = '2-Central' # Illinois region
FEATURES_SELECTION_QUANTILE = .75 # upper quantile used with the statistical-based feature selection

Download spatial files locally.

In [ ]:
download_s3_folder(AWS_S3_BUCKET,SPATIAL_FILES_KEY, "tmp/spatial-files")
download_s3_folder(AWS_S3_BUCKET,SENTINEL_2_SHAPEFILE_KEY, "tmp/Sentinel-2-Shapefile-Index")

Read the dataset that includes the output of the simulations at the field level (`id_10` and `id_field`).

In [ ]:
data = pd.read_csv(
    f"s3://{AWS_S3_BUCKET}/{KEY_YIELD_CURVE}",
    index_col=0)

Read the `geojson` file to recover the cell IDs selected in `00 Geospatial Processing.ipynb`. This file includes FIPS, COUNTIES, CELLS, and the FIELDS mapping.

In [ ]:
gp_cells_ids = gpd.read_file("tmp/zonal_fips_stats.geojson")
cells_ids = list(gp_cells_ids.id_10.unique())

In [ ]:
# filter by YEAR and REGION
df = data[(data.year == YEAR) & (data.region == CROP_REGION)]

# filter by the selected cells IDs
df = df[df.id_10.isin(cells_ids)]

### Read zonal statistics

> **Note**: Files produced in the `00 Geospatial Processing.ipynb` notebook.

In [ ]:
df_stats = dd.read_csv(
    f"s3://{AWS_S3_BUCKET}/data/zonal-statistics-allbands/"
    f"{CROP_TYPE}/{YEAR}/*/*",
    dtype={"id_10": object},
    include_path_column = True
).compute()

df_stats['crop_type'] = 'corn'

In [ ]:
# Retrieve isoweek from path
df_stats['isoweek'] = df_stats['path'].str.split('/').str[-2].str.split("-").str[-1].astype(int)

In [ ]:
cols = "mean|id_10|FIPS|crop_type|isoweek"

# Select the mean statistics
df_stats_mean = df_stats[df_stats.columns[df_stats.columns.str.contains(cols)]]

# Drop rows with inf, -inf and nan values
df_stats_mean = df_stats_mean.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

df_stats_mean['id_10'] = df_stats_mean['id_10'].astype(int)

In [ ]:
numerical_cols = list(df_stats_mean.columns[df_stats_mean.columns.str.contains("mean")])

# Rescale the numerical features
scaler = RobustScaler()
df_stats_mean[numerical_cols] = scaler.fit_transform(df_stats_mean[numerical_cols])

### SVIs signature variations (Grouped by isoweek)

In [ ]:
for crop in df_stats_mean.crop_type.unique():
    for week in df_stats_mean.isoweek.unique():
        df_px = df_stats_mean[(df_stats_mean.isoweek == week) & (df_stats_mean.crop_type == crop)] \
        .groupby(by=["FIPS"])[numerical_cols].mean().T
        
        columns = df_px.columns
        
        fig = px.line(df_px,
                      x=df_px.index,
                      y=columns,
                      line_shape="spline",
                      render_mode="svg",
                      title=f"week: {week}; crop type: {crop}")
        fig.show()

### SVIs signature variations (Grouped by FIPS)

In [ ]:
for crop in df_stats_mean.crop_type.unique():
    for fips in df_stats_mean.FIPS.unique():
        df_px = df_stats_mean[(df_stats_mean.FIPS == fips) & (df_stats_mean.crop_type == crop)] \
        .groupby(by=["isoweek"])[numerical_cols].mean().T
        
        columns = df_px.columns
        
        fig = px.line(df_px,
                      x=df_px.index,
                      y=columns,
                      line_shape="spline",
                      render_mode="svg",
                      title=f" FIPS: {fips}; crop type: {crop}")
        fig.show()

### Reshape data frame and create a mapping file for the corn phenology cycle

In [ ]:
# Reshape the frame to convert [isoweek, crop_type] elements into individual columns
df_stats_mean_groups = df_stats_mean.groupby(['isoweek','crop_type'])
week_groups = []

for name, group in df_stats_mean_groups:
    group = group.rename(columns={c: f'{c}_{str(name[1])}_{str(name[0])}' for c in group.columns if c in numerical_cols})
    week_groups.append(group)
    
df_stats_mean_pivoted = pd.concat(week_groups,axis=1)
df_stats_mean_pivoted = df_stats_mean_pivoted.loc[:, ~df_stats_mean_pivoted.columns.duplicated()]
df_stats_mean_pivoted = df_stats_mean_pivoted.drop(columns={'isoweek'})

In [ ]:
df_merged = pd.merge(df,df_stats_mean_pivoted, on='id_10')
df_merged = df_merged.drop(columns = {'Y_corn_lt_avg','region','year','lat','long'})

#### Define the crop phenology stages for **CORN** according to [database characterization](https://www.sciencedirect.com/science/article/pii/S2352340921010283#tbl0001)

Within the following section, we learn the crop phenology graph (DAG), which is a collection of nodes and edges, where the `nodes` are various indicators of crop growth, soil characteristics, atmospheric conditions, and `edges` between them represent temporal-causal relationships. `Parent nodes` are the field-related parameters (including the day of sowing and area planted), whereas the `child nodes` are the yield, nitrogen uptake, and nitrogen leaching targets.

We now construct a mapping file that assigns a **level** index to each variable that corresponds to a crop phenology stage.

Corn phenology cycle

* Vegetative stages
    * VE – emergence, coleoptile breaks through the soil surface

    * V1 - one leaf collar is visible

    * V2 - second leaf collar is visible

    * V3 - third leaf collar is visible, plant begins to photosynthesize and rely on nodal root system

    * V4 - fourth leaf collar is visible

    * V5-V6 - fifth to sixth leaf collars are visible, growing point is above the soil surface, critical period of nitrogen uptake begins, and kernel row numbers are determined
    * V7-V(n) - seventh to nth leaf collars are visible, period of very rapid growth

    * VT – Tasselling – tassel is emerged, transitioning to reproductive phase
    
* Reproductive stages
    * R1 Silking – silks emerge from husks

    * R2 Blister – kernels are white on outside and inner fluid is clear

    * R3 Milk - kernels are yellow on the outside and inner fluid is milky-white

    * R4 Dough - milky inner fluid thickens from starch accumulation

    * R5 Dent – more than 50% of kernels are dented

    * R6 Physiological maturity – black layer formed

In [ ]:
CHILD_LIST = ['P', 'sand_40cm', 'om_40cm', 'clay_40cm', 'dul_dep',
              'day_sow', 'll15_dep', 'restriction', 'whc', 'id_10', 'station', 'id_field', 'FIPS']
PARENTS_LIST = ['Y_corn', 'n_uptake', 'LAI_max',
                'rain_annual', 'Y_soy', 'L1', 'L2', 'L']


def staging(row):

    postfix = row.split('_')[-1]

    # Available at planting
    if row in CHILD_LIST:
        return 0

    # Available at harvest
    elif row in PARENTS_LIST:
        return 5
    
    # N fertiliser
    elif row == 'N_fert':
        return 1

    # v5
    elif postfix == 'v5':
        return 2

    # R6 - R8
    elif postfix == 'fw':
        return 3

    elif postfix.isdigit():
        # 1 Jan. to planting
        if int(postfix) == 1:
            return 0

        # planting to v5
        elif int(postfix) == 2:
            return 1

        # v5 - R1
        elif int(postfix) == 3:
            return 2

        # R1 - R3
        elif int(postfix) == 4:
            return 3

        # R3 - R6
        elif int(postfix) == 5:
            return 4

        # harvest - Dec 31
        elif int(postfix) == 6:
            return -1

        # v5 - R1
        elif int(postfix) >= 20 and int(postfix) < 26:
            return 2

        # R1 - R3
        elif int(postfix) >= 26 and int(postfix) < 33:
            return 3

        # R3 - R6
        elif int(postfix) >= 33:
            return 4

        else:
            return -1

    else:
        return -1


df_mapping = pd.DataFrame(data=df_merged.columns, columns=['variable'])
df_mapping['level'] = df_mapping.apply(
    lambda row: staging(row['variable']), axis=1)

In [ ]:
REGION = CROP_REGION.replace("-","_")

df_merged.to_csv(
    f"s3://{AWS_S3_BUCKET}/data/enhanced/"
    f"enhanced_dataset_{YEAR}_{REGION}.csv",
    index=False,
    )

df_mapping.to_csv(
    f"s3://{AWS_S3_BUCKET}/data/enhanced/"
    f"stage_mapping_{YEAR}_{REGION}.csv",
    index=False,
    )

## Multicollinearity study

Compute the correlation matrix and the [variance inflation factor](https://www.statsmodels.org/dev/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html), for each **corn** phenology stage.

In [ ]:
df_eda = df_mapping[~df_mapping.variable.isin(
    ['id_10', 'station', 'id_field', 'FIPS'])]

for level in sorted(df_eda.level.unique()):

    if level == -1:
        continue

    print(f" Correlation Matrix for Stage: {level}")

    level_mapping = df_eda[df_eda.level == level]
    cols = list(level_mapping.variable.unique())

    corr = df_merged[cols].corr()

    heatmap_pandas(corr)

    print("\n")
    print(f"Variance Inflation Factor (VIF) for Stage: {level}")

    vif = pd.DataFrame()
    vif["features"] = cols
    vif["vif"] = [variance_inflation_factor(
        df_merged[cols].values, i) for i in range(df_merged[cols].shape[1])]
    vif = vif.set_index('features')
    vif = vif.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
    display(vif.style.apply(centered_gradient, cmap=get_cmap('Reds')))

    print("\n")
    print("\n")

## Feature selection 

Compute global feature importance using [SHAP (SHapley Additive exPlanations)](https://github.com/slundberg/shap), for each **corn** phenology stage.

In [ ]:
selected_features_corn = []

for level in sorted(df_mapping.level.unique()):

    if level == -1:
        continue

    print(f" [CORN] Shap Analysis for Stage: {level}")

    level_mapping = df_mapping[df_mapping.level == level]
    cols = list(level_mapping.variable.unique())

    if 'Y_corn' in cols:
        cols.remove('Y_corn')

    # Select predictors
    X = df_merged[cols]

    # Select target
    y = df_merged['Y_corn']

    # Train XGBRegressor
    model = xgboost.XGBRegressor().fit(X, y)

    # Compute SHAP feature importance
    feature_importance = global_shap_importance(model, X)

    feature_importance['importance_scaled'] = (
        feature_importance['importance'].values - feature_importance['importance'].values.min()) \
        / (feature_importance['importance'].values - feature_importance['importance'].values.min()
           ).sum()

    # Filter by the x quantile
    quantile = feature_importance['importance_scaled'].quantile(
        FEATURES_SELECTION_QUANTILE)
    feature_importance = feature_importance.loc[(
        feature_importance['importance_scaled'] > quantile)].sort_values(by=['importance_scaled'], ascending=False)

    selected_features_corn.append(list(feature_importance.features.unique()))

    feature_importance = feature_importance.set_index('features')

    display(feature_importance.style.apply(
        centered_gradient, cmap=get_cmap('Greens')))
    
    print("\n")
    print("\n")

### Save the filtered dataset and the crop staging mapping files to S3

In [ ]:
satellite_images = [
    feat for feat in df_merged.columns if feat.startswith('mean_')]

selected_features = sorted(list(set(sum(selected_features_corn, [
]) + satellite_images + ['Y_corn', 'FIPS', 'id_10', 'id_field'])))

# save selected features
df_merged_filtered = df_merged[selected_features]
df_mapping_filtered = df_mapping[df_mapping.variable.isin(selected_features)]


In [ ]:
REGION = CROP_REGION.replace("-","_")

df_merged_filtered.to_csv(
    f"s3://{AWS_S3_BUCKET}/data/enhanced/"
    f"enhanced_dataset_filtered_{YEAR}_{REGION}.csv",
    index=False,
    )

df_mapping_filtered.to_csv(
    f"s3://{AWS_S3_BUCKET}/data/enhanced/"
    f"stage_mapping_filtered_{YEAR}_{REGION}.csv",
    index=False,
    )

### Next Steps

You can now open [02 Causal Model.ipynb](02%20Causal%20Model.ipynb) and follow the steps inside the notebook.